In [34]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from bs4 import BeautifulSoup
import pickle
from RecFunctions import get_products_by_type,get_products_by_problem, categorical_columns, type_column

# Functions Scrape Data and Get Info Df's For SkinStore

In [35]:
cleansers=get_products_by_type('cleansers',14)
exfoliators=get_products_by_type('exfoliators',6)
removers=get_products_by_type('makeup-removers',3)
toners=get_products_by_type('toners',6)
mists=get_products_by_type('mists',3)
treatments=get_products_by_type('treatments',14)
serums=get_products_by_type('serums',16)
lotions=get_products_by_type('lotions',6)
moisturizers=get_products_by_type('moisturizers',23)
balms=get_products_by_type('balms',3)
oils=get_products_by_type('oils',5)
masks=get_products_by_type('masks',9)
peels=get_products_by_type('peels',3)
lips=get_products_by_type('lip-care',4)
eyes=get_products_by_type('eye-care',9)
supplements=get_products_by_type('supplements',1)
tools=get_products_by_type('tools',5)

In [36]:
acnedf,acne = get_products_by_problem('acne-blemishes',10)
agedf,age = get_products_by_problem('anti-aging',30)
darkcdf,darkcircles = get_products_by_problem('dark-circles',5)
drydf,dryness = get_products_by_problem('dry-skin',19)
ncdf,norm= get_products_by_problem('normal-combination',17)
oilydf,oily=get_products_by_problem('oily-skin',17)
sensitivedf,sensi=get_products_by_problem('sensitive-skin',17)
reddf,redness=get_products_by_problem('redness-rosacea',17)
# images=[]
# images=(reddf.image+sensitivedf.image+drydf.image+oilydf.image+ncdf.image+darkcdf.image+agedf.image+acnedf.image)

In [37]:
braa=[]
ratee=[]
pri=[]
ur=[]
imgs=[]
for i,r,p,u in zip(acnedf.prodName,acnedf.rating,acnedf.price,acnedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(agedf.prodName,agedf.rating,agedf.price,agedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(darkcdf.prodName,darkcdf.rating,darkcdf.price,darkcdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(drydf.prodName,drydf.rating,drydf.price,drydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(ncdf.prodName,ncdf.rating,ncdf.price,ncdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(oilydf.prodName,oilydf.rating,oilydf.price,oilydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(sensitivedf.prodName,sensitivedf.rating,sensitivedf.price,sensitivedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(reddf.prodName,reddf.rating,reddf.price,reddf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
datadict=[]
# for i,r,p,u,img in zip(braa,ratee,pri,ur,imgs):
#     datadict.append({'prodName':i,'rating':r,'price':p,'url':u,'image':img})
for i,r,p,u in zip(braa,ratee,pri,ur):
    datadict.append({'prodName':i,'rating':r,'price':p,'url':u})
finaldf=pd.DataFrame(datadict)

In [39]:
finaldf.rating=[float(i) for i in finaldf.rating]
finaldf.price=[float(i) for i in finaldf.price]

In [40]:
see=finaldf.set_index('prodName')

In [42]:
finaldf

,price,prodName,rating,url
0,29.99,La Roche-Posay Effaclar Adapalene Gel 0.1% Ret...,5.00,12190079
1,85.00,Slip Silk Pillowcase - Queen - White,5.00,11561755
2,105.00,Slip Silk Pillowcase King - White,5.00,11561765
3,85.00,Slip Silk Pillowcase - Queen - Charcoal,4.69,11561753
4,85.00,Slip Silk Pillowcase - Queen - Silver,5.00,11561759
5,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,4.11,11289619
6,85.00,Slip Silk Pillowcase - Queen - Pink,5.00,11561754
7,67.00,SkinCeuticals Phyto Corrective Gel,4.91,11289629
8,85.00,Slip Silk Pillowcase - Queen - Caramel,5.00,11561752
9,29.50,Paula's Choice Skin Perfecting 2% BHA Liquid E...,4.75,11174178


In [43]:
# with open('pickles/findf.pickle', 'wb') as f:
#     pickle.dump(finaldf, f, pickle.HIGHEST_PROTOCOL)

## Finalize Full DF

In [44]:
categorical_columns('age',age,finaldf)
categorical_columns('darkcircles',darkcircles,finaldf)
categorical_columns('acne',acne,finaldf)
categorical_columns('dry',dryness,finaldf)
categorical_columns('redness',redness,finaldf)
categorical_columns('sensitive',sensi,finaldf)
categorical_columns('oily',oily,finaldf)
categorical_columns('normal',norm,finaldf)
categorical_columns('cleanser',cleansers,finaldf)
categorical_columns('exfoliator',exfoliators,finaldf)
categorical_columns('makeup-removers',removers,finaldf)
categorical_columns('toner',toners,finaldf)
categorical_columns('mist',mists,finaldf)
categorical_columns('treatment',treatments,finaldf)
categorical_columns('serum',serums,finaldf)
categorical_columns('lotion',lotions,finaldf)
categorical_columns('moisturizer',moisturizers,finaldf)
categorical_columns('balm',balms,finaldf)
categorical_columns('oil',oils,finaldf)
categorical_columns('mask',masks,finaldf)
categorical_columns('peel',peels,finaldf)
categorical_columns('lip',lips,finaldf)
categorical_columns('eye',eyes,finaldf)
categorical_columns('supplement',supplements,finaldf)
categorical_columns('tool',tools,finaldf)

In [92]:
with open('pickles/CategoricalItemDF.pickle', 'rb') as f:
    df = pickle.load(f)

In [28]:
# with open('FINALDFurl.pickle', 'wb') as f:
#     pickle.dump(finaldf, f, pickle.HIGHEST_PROTOCOL)

## Get Review Info & Get Into DF

In [114]:
ratingdict=[]
for u in finaldf.url:
    response=requests.get('https://www.skinstore.com/the-ordinary-aha-30-bha-2-peeling-solution-30ml/{}.html'.format(u))
    soup=BeautifulSoup(response.content,'html.parser')
    titles=soup.findAll('h3',{'class':'productReviews_topReviewTitle'})
    ratings=soup.findAll('div',{'class':'productReviews_topReviewsRatingStarsContainer'})
    contents=soup.findAll('p',{'class':'productReviews_topReviewsExcerpt'})
    dates=soup.findAll('span',{'data-js-element':'createdDate'})
    users=soup.findAll('div',{'class':'productReviews_footerDateAndName'})
    brands=soup.find('div',{'data-information-component':'brand'})
    products=soup.find('h1',{'data-product-name':'title'})
    for t,r,c,d,i in zip(titles,ratings,contents,dates,users):
        ratingdict.append({'url':u,'brandName':brands.text.replace('\n',''),'prodName':products.text,'title':t.text.replace('\n',''),'rating':str(r).split('aria-label=')[1][1:2],'content':c.text.replace('\n','').replace('\r',''),'date':d.text,'user':i.text.replace('\n','').split('by')[1].lower()})
ratingdf=pd.DataFrame(ratingdict)
ratingdf['date']=pd.to_datetime(ratingdf.date)
ratingdf['rating']=[int(r) for r in ratingdf['rating']]
ratingdf.user=ratingdf.user.replace('','user')

In [117]:
ratingdf.drop_duplicates(inplace=True)
ratingdf.content=[c.lower() for c in ratingdf.content]
ratingdf

,brandName,content,date,prodName,rating,title,url,user
0,Slip,the silk cases are great. i don’t wake up with...,2019-10-28,Slip Silk Pillowcase - Queen - White,5,No more face marks,11561755,dsm
1,Slip,these pillow cases are buttery smooth - i've g...,2019-09-20,Slip Silk Pillowcase - Queen - White,5,Worth the money,11561755,livie
2,Slip,i've been using slip silk pillowcases for almo...,2019-05-15,Slip Silk Pillowcase - Queen - White,5,A MUST have,11561755,gina
3,Slip,"... pillowcase, it makes such a difference and...",2018-12-21,Slip Silk Pillowcase - Queen - White,5,My new favorite,11561755,robin
4,Slip,slipsilk silk pillowcases are an excellent pro...,2018-08-20,Slip Silk Pillowcase - Queen - White,5,Silk pillowcase,11561755,silk case
5,Slip,love this for hair and even more for my face. ...,2018-10-12,Slip Silk Pillowcase King - White,5,Better than expected,11561765,lw
6,Slip,it is super soft &a comfortable. just make you...,2017-11-08,Slip Silk Pillowcase - Queen - Charcoal,5,Love it,11561753,lv lover elaine
7,SkinCeuticals,this ha serum w/b5 gel is the best! i use it ...,2019-11-01,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,5,the best,11289619,ha b5
8,SkinCeuticals,this product is very hydrating and there is a ...,2019-10-31,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,5,Love it!,11289619,123lees
9,SkinCeuticals,based on my dermatologist's recommendation alm...,2018-06-10,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,5,I've used this for close to 2 decades paired w...,11289619,mitch


In [118]:
ratingdf['brandName']=[r.replace('\n','') for r in ratingdf['brandName']]

In [119]:
finaldf.drop(columns='rating',inplace=True)

KeyError: "['rating'] not found in axis"

In [78]:
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 28 columns):
price              1912 non-null float64
prodName           1912 non-null object
url                1912 non-null object
age                1912 non-null int64
darkcircles        1912 non-null int64
acne               1912 non-null int64
dry                1912 non-null int64
redness            1912 non-null int64
sensitive          1912 non-null int64
oily               1912 non-null int64
normal             1912 non-null int64
cleanser           1912 non-null int64
exfoliator         1912 non-null int64
makeup-removers    1912 non-null int64
toner              1912 non-null int64
mist               1912 non-null int64
treatment          1912 non-null int64
serum              1912 non-null int64
lotion             1912 non-null int64
moisturizer        1912 non-null int64
balm               1912 non-null int64
oil                1912 non-null int64
mask               1912 non-nul

In [125]:
final=pd.merge(finaldf,ratingdf, on='url')

In [126]:
final.drop(columns=['prodName_y'],inplace=True)
final.rename(columns={'prodName_x':'prodName'},inplace=True)

In [127]:
final

,price,prodName,url,age,darkcircles,acne,dry,redness,sensitive,oily,...,lip,eye,supplement,tool,brandName,content,date,rating,title,user
0,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,the silk cases are great. i don’t wake up with...,2019-10-28,5,No more face marks,dsm
1,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,these pillow cases are buttery smooth - i've g...,2019-09-20,5,Worth the money,livie
2,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,i've been using slip silk pillowcases for almo...,2019-05-15,5,A MUST have,gina
3,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,"... pillowcase, it makes such a difference and...",2018-12-21,5,My new favorite,robin
4,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,slipsilk silk pillowcases are an excellent pro...,2018-08-20,5,Silk pillowcase,silk case
5,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,the silk cases are great. i don’t wake up with...,2019-10-28,5,No more face marks,dsm
6,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,these pillow cases are buttery smooth - i've g...,2019-09-20,5,Worth the money,livie
7,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,i've been using slip silk pillowcases for almo...,2019-05-15,5,A MUST have,gina
8,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,"... pillowcase, it makes such a difference and...",2018-12-21,5,My new favorite,robin
9,85.00,Slip Silk Pillowcase - Queen - White,11561755,0,0,1,0,1,1,0,...,0,0,0,0,Slip,slipsilk silk pillowcases are an excellent pro...,2018-08-20,5,Silk pillowcase,silk case


In [129]:
with open('pickles/df1.pickle', 'wb') as f:
    pickle.dump(final, f, pickle.HIGHEST_PROTOCOL)

In [38]:
# with open('recdf.pickle', 'wb') as f:
#     pickle.dump(omgdf, f, pickle.HIGHEST_PROTOCOL)

In [33]:
# with open('userratingsDF.pickle', 'wb') as f:
#     pickle.dump(ratingdf, f, pickle.HIGHEST_PROTOCOL)